In [33]:
import pandas as pd
pd.set_option('display.max_columns', 50)

In [34]:
data = pd.read_csv('heart_num_0228.csv')
print(len(data))
data.head()

274034


,State,Sex,GeneralHealth,PhysicalHealthDays,MentalHealthDays,LastCheckupTime,PhysicalActivities,SleepHours,RemovedTeeth,HadHeartAttack,HadAngina,HadStroke,HadAsthma,HadSkinCancer,HadCOPD,HadDepressiveDisorder,HadKidneyDisease,HadArthritis,HadDiabetes,DeafOrHardOfHearing,BlindOrVisionDifficulty,DifficultyConcentrating,DifficultyWalking,DifficultyDressingBathing,DifficultyErrands,SmokerStatus,ECigaretteUsage,ChestScan,AgeCategory,HeightInMeters,WeightInKilograms,BMI,AlcoholDrinkers,HIVTesting,FluVaxLast12,PneumoVaxEver,HighRiskLastYear,CovidPos,"RaceEthnicityCategory_Black only, Non-Hispanic",RaceEthnicityCategory_Hispanic,"RaceEthnicityCategory_Multiracial, Non-Hispanic","RaceEthnicityCategory_Other race only, Non-Hispanic","RaceEthnicityCategory_White only, Non-Hispanic","TetanusLast10Tdap_No, did not receive any tetanus shot in the past 10 years","TetanusLast10Tdap_Yes, received Tdap","TetanusLast10Tdap_Yes, received tetanus shot but not sure what type","TetanusLast10Tdap_Yes, received tetanus shot, but not Tdap"
0,0,0,4.0,0.0,0.0,3.0,0,6.0,3.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,3.0,3.0,0,80,1.60,68.04,26.57,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0
1,0,0,1.0,2.0,0.0,3.0,1,9.0,3.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.0,3.0,1,40,1.57,53.98,21.77,1,0,0,1,0,0,0,0,0,0,1,1,0,0,0
2,0,1,0.0,1.0,0.0,3.0,0,7.0,3.0,1,0,1,0,0,0,0,0,0,2,0,0,0,0,0,0,3.0,3.0,0,80,1.80,84.82,26.08,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0
3,0,0,3.0,0.0,0.0,3.0,1,7.0,3.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,3.0,0,80,1.65,62.60,22.96,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0
4,0,0,2.0,0.0,0.0,3.0,0,8.0,3.0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,3.0,3.0,1,80,1.63,73.48,27.81,0,0,1,1,0,0,0,0,0,0,1,0,0,1,0


In [35]:
data.isna().sum()

State                                                                          0
Sex                                                                            0
GeneralHealth                                                                  0
PhysicalHealthDays                                                             0
MentalHealthDays                                                               0
LastCheckupTime                                                                0
PhysicalActivities                                                             0
SleepHours                                                                     0
RemovedTeeth                                                                   0
HadHeartAttack                                                                 0
HadAngina                                                                      0
HadStroke                                                                      0
HadAsthma                   

In [36]:
Q1 = data[['SleepHours', 'BMI']].quantile(q = 0.25)
Q3 = data[['SleepHours', 'BMI']].quantile(q = 0.75)
print(Q1)
print()
print(Q3)

SleepHours     6.00
BMI           24.21
Name: 0.25, dtype: float64

SleepHours     8.00
BMI           31.89
Name: 0.75, dtype: float64


In [37]:
IQR = Q3-Q1

In [38]:
data = data[(data['SleepHours'] <= Q3['SleepHours']+IQR['SleepHours']*1.5)&(data['SleepHours'] >= Q1['SleepHours']-IQR['SleepHours']*1.5)]
data = data[(data['BMI'] <= Q3['BMI']+IQR['BMI']*1.5)&(data['BMI'] >= Q1['BMI']-IQR['BMI']*1.5)]
print(len(data))

261812


In [39]:
X = data.drop('HadHeartAttack', axis = 1)
y = data['HadHeartAttack']

In [41]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1234)

In [42]:
from sklearn.preprocessing import Normalizer
norm = Normalizer()
X_train_scaled = norm.fit_transform(X_train)
X_test_scaled = norm.transform(X_test)

In [45]:
from sklearn.linear_model import SGDClassifier
sgd = SGDClassifier(max_iter = 10000, random_state = 1234)
sgd.fit(X_train, y_train)

SGDClassifier(max_iter=10000, random_state=1234)

In [46]:
from sklearn.metrics import accuracy_score
sgd_pred = sgd.predict(X_test)
sgd_acc = accuracy_score(y_test, sgd_pred)
print(f'SGDClassifier Accuracy: {sgd_acc*100:.2f}%')

SGDClassifier Accuracy: 93.69%


In [47]:
from sklearn.ensemble import AdaBoostClassifier
clf = AdaBoostClassifier(n_estimators = 100, random_state = 1234)
clf.fit(X_train, y_train)

AdaBoostClassifier(n_estimators=100, random_state=1234)

In [49]:
clf_pred = clf.predict(X_test)
clf_acc = accuracy_score(y_test, clf_pred)
print(f'AdaBoost Accuracy: {clf_acc*100:.2f}%')

AdaBoost Accuracy: 94.54%


In [50]:
from imblearn.over_sampling import SMOTE

In [52]:
import imblearn
imblearn.__version__

'0.11.0'

In [53]:
import sklearn
sklearn.__version__

'1.3.0'